Generate Paired Data for Denoising

In [2]:
import os
import numpy as np
from PIL import Image
import random

def add_noise(image, noise_level=50):
    """
    Adds Gaussian noise to an image.
    Args:
        image (PIL.Image): Input image.
        noise_level (int): Standard deviation of the Gaussian noise.
    Returns:
        PIL.Image: Noisy image.
    """
    np_image = np.array(image).astype(np.float32)
    noise = np.random.normal(0, noise_level, np_image.shape)
    noisy_image = np_image + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy_image)

# Directories for saving
os.makedirs('./paired_data/input', exist_ok=True)
os.makedirs('./paired_data/target', exist_ok=True)

# Generate paired data
data_dir = './clean_images'  # Replace with the directory of your clean images
for i, file_name in enumerate(os.listdir(data_dir)):
    image_path = os.path.join(data_dir, file_name)
    image = Image.open(image_path).convert("RGB")

    noisy_image = add_noise(image, noise_level=random.randint(15, 70))  # Add random noise level

    noisy_image.save(f'./paired_data/input/noisy_{i}.jpg')
    image.save(f'./paired_data/target/clean_{i}.jpg')

print("Paired noisy-clean dataset created!")


Paired noisy-clean dataset created!
